# 一、中原爬蟲：

### step1.引入套件&設定環境

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import numpy as np
import os 
import time 
import pandas as pd
import urllib
import math
#math.ceil() 無條件進位
#math.floor() 無條件捨去
#round() 四捨五入

In [16]:
os.chdir('C:/Users/chiaming/Desktop/python')

In [3]:
driver = webdriver.Chrome(executable_path="chromedriver.exe")
driver.get("https://www.com.tw/cross/")
school_link = ['中原大學','元智大學','東吳大學','東海大學','長庚大學','國立東華大學','國立臺中教育大學','淡江大學','逢甲大學','臺北市立大學','輔仁大學','世新大學','實踐大學','銘傳大學']


### step2.爬取十四校108年資料

In [4]:
time_start = time.time()
for first_url in school_link:
    WebDriverWait(driver, 10).until(lambda d: d.find_element_by_link_text(first_url)).click()
    print("【"+first_url+"】")
    x = BeautifulSoup(driver.page_source,'lxml')
    dept_html = x.select('tbody > tr > td:nth-of-type(2) a[href^="check_0"]')
    university_index = []
    university_name = []
    university_region = []
    university_school = []
    university_just_school = []
    university_just_dept = []
    university_enroll = []
    university_inroll = []
    for second_url in range(len(dept_html)):
        print(">>第"+str(second_url+1)+"個系")
        driver.get("https://www.com.tw/cross/"+dept_html[second_url]['href'])
        count = 0
        person_school = []
        person_just_school = []
        person_just_dept = []
        person_enroll = []
        person_index = []
        person_region = []
        person_name = []
        person_inroll = []
        person = BeautifulSoup(driver.page_source,'lxml')
        only_person_index = [ index.text.strip() for index in person.select('tbody:nth-of-type(3) > tr["bgcolor"] > td:nth-of-type(2) div') ]
        person_len = len(only_person_index)
        for third_url in range(person_len):
            #學校、學校正備取
            a = person.select('tbody:nth-of-type(3) > tr:nth-of-type('+str(third_url+4)+') > td:nth-of-type(5) tbody > tr > td:nth-of-type(2)')
            b = person.select('tbody:nth-of-type(3) > tr:nth-of-type('+str(third_url+4)+') > td:nth-of-type(5) tbody > tr > td:nth-of-type(3)')
            only_person_region = [region.text.strip().replace("考區 : ","") for region in person.select('tbody:nth-of-type(3) > tr:nth-of-type('+str(third_url+4)+') > td:nth-of-type(3) a')]
            only_person_name = [ name.text.strip() for name in person.select('tbody:nth-of-type(3) > tr:nth-of-type('+str(third_url+4)+') > td:nth-of-type(4)')]
            
            inroll = []
            inro = person.select('tbody:nth-of-type(3) > tr:nth-of-type('+str(third_url+4)+') > td:nth-of-type(5) tbody > tr > td:nth-of-type(1) > div:nth-of-type(1)')
            for i in inro:
                if i.select('img') != []:
                    inroll.append('分發錄取')
                else:
                    inroll.append('')
            cou = 0 #算分發錄取用
            
            for i,k in zip(a,b):
                if i.text.strip() == "" and k.text.strip() == "":
                    cou +=1
                    continue
                else:
                    person_inroll.append(inroll[cou])
                    cou +=1
                    word = i.text.strip().replace("\n","").replace('中原大學中原大學與美國天普大學商學管理雙學士學位學程','中原大學美國天普商學管理雙學士學位學程').replace('大學 ','大學').replace('大學','大學 ').replace('技術學院','技術學院 ').replace('健康學院','健康學院 ').replace('中信金融管理學院','中信金融管理學院 ').replace('稻江科技暨管理學院','稻江科技暨管理學院 ').replace("醫學院","醫學院 ").replace("基督學院","基督學院 ")
                    person_school.append(word)
                    person_enroll.append(k.text.strip().replace("\n",""))
                    person_just_school.append(word.split(" ")[0])
                    person_just_dept.append(word.split(" ")[1])
                    person_index.append(only_person_index[count])
                    person_region.append(only_person_region[0])
                    person_name.append(only_person_name[0])
            count +=1 
        dept = pd.DataFrame({'排名':person_index,'考區':person_region,'姓名':person_name,'分發錄取':person_inroll,'申請學校':person_school,'學校':person_just_school,'系所':person_just_dept,'正錄取':person_enroll})
        dept.to_excel(a[0].text.strip().replace('\n','').replace('/','')+'.xlsx',sheet_name=a[0].text.strip().replace('\n','').replace('/',''))
        university_index += person_index
        university_name += person_name
        university_region += person_region
        university_school += person_school
        university_just_school += person_just_school
        university_just_dept += person_just_dept
        university_enroll += person_enroll
        university_inroll += person_inroll
        WebDriverWait(driver, 10).until(lambda d: d.find_element_by_link_text('回其他學系')).click()
    univer = pd.DataFrame({'排名':university_index,'考區':university_region,'姓名':university_name,'分發錄取':university_inroll,'申請學校':university_school,'學校':university_just_school,'系所':university_just_dept,'正錄取':university_enroll})
    univer.to_excel(first_url.replace('/','')+'.xlsx',sheet_name=first_url.replace('/',''))
    WebDriverWait(driver, 10).until(lambda d: d.find_element_by_link_text('回一般大學')).click()

time_end = time.time()
print(time_end-time_start)

【中原大學】
>>第1個系
>>第2個系
>>第3個系
>>第4個系
>>第5個系
>>第6個系


KeyboardInterrupt: 

### step3.各校資料表所有系合併
* 這裡會跑十四次 (因為一校就要合併所有系成為一個大資料表一次)

In [3]:
all_data = pd.DataFrame()
for i in range(len(school_link)):
    globals()['col'+str(i+1)] = pd.read_excel("C:/Users/chiaming/Desktop/中原十四校(全)/"+school_link[i]+'.xlsx')
    all_data = all_data.append(globals()['col'+str(i+1)])

# 二、資料庫分析：

### step1.讀各系資料表進來

In [3]:
os.chdir('C:/Users/chiaming/Desktop/中原十四校/中原108/商學院/原始資料')

In [2]:
science_name = ['中原大學 企業管理學系服務業管理組','中原大學 企業管理學系高科技業管理組','中原大學 企業管理學系工商管理組','中原大學 國際經營與貿易學系','中原大學 會計學系','中原大學 資訊管理學系','中原大學 財務金融學系','中原大學 美國天普商學管理雙學士學位學程']
science_path = [i.replace('中原大學 ','中原大學') for i in science_name]

In [4]:
for file in range(len(science_path)):
    globals()['file'+str(file+1)] = pd.read_excel(science_path[file]+'.xlsx')

In [ ]:
commerce = pd.DataFrame()
for i in range(len(science_name)):
    commerce = commerce.append(region_count(eval('file'+str(i+1))))

### step2.建立資料清洗函式
* 抓指考仔函式
* 分析各系就讀&放棄中原人數，並輸出成 [流入] [流出] 資料表函式
* 這邊要手工調的部分是 讀取資料用函式時，要去改路徑 (在step.1)

#### >>抓指考仔函式

In [878]:
def find_not(file):
    count = 0
    region = file.iloc[-1,-1]
    for i in range(region):
        check = file[file['區域']==i+1]
        if(check['分發錄取'].isna().sum() == len(check)):
            count +=1
            print('指考仔區域:'+str(i+1))
    print("指考人數:"+str(count))

#### >>分析函式

In [1204]:
##算區域個數
def region_counter(file,pos):
    file = file[file['排名'].str.contains('原|離|金|澎|連')==False]
    
    print('----------------------------------------------')
    print('【'+pos+'】')
    last = file[file['分發錄取'].isin(['分發錄取']) & file['申請學校'].isin([pos])].index[-1]
    
    region_contain = []
    region = 0
    check = file.iloc[0,1]
    name = file.iloc[0,2]
    for row in range(len(file)):
        if((check,name) == (file.iloc[row,1],file.iloc[row,2])):
            region+=1
        else:
            check = file.iloc[row,1]
            name = file.iloc[row,2]
        #print(region)
            region_contain.append(region)
            region=1

    ##標記區域
    index_col = []
    count = 0
    for i in region_contain:
        count += 1
        for row in range(i):
            index_col.append(count)
    
    tail = index_col[-1] 
    for i in range(len(file)-len(index_col)):
        index_col.append(tail+1)
    
    file['區域'] = index_col 
    
    for i in range(20):
        try:
            if(tuple(list(file.loc[last,['考區','姓名']].values)) != tuple(list(file.loc[(last+i),['考區','姓名']].values))):
                x = last+i-1
                break
        except:
            x = last+i-1
            break
    print('最後一個錄取: '+str(x))
    real = file.loc[:x,:]
    
    find_not(real)
    
    find_drop = real[(real['分發錄取'].isna() == False) & (real['申請學校'].isin([pos])==False)]
    find_in =  real[(real['分發錄取'].isna() == False) & (real['申請學校'].isin([pos])==True)]
    
    print('共有幾人: '+str(real.drop_duplicates(['排名']).shape[0]))
    print('放棄: '+str(find_drop.drop_duplicates(['排名']).shape[0]))
    print('入學: '+str(find_in.shape[0]))
    
    ### 列篩選 isin (丟list複合篩選)
    find_EN = find_drop.iloc[:,0:3].values
    l = [tuple(i) for i in find_EN]
    drop1 = file[file[['排名','考區', '姓名']].apply(tuple, axis = 1).isin(l)]
    
    dept = drop1[drop1['分發錄取']=='分發錄取']['申請學校'].value_counts().index
    value = drop1[drop1['分發錄取']=='分發錄取']['申請學校'].value_counts().values
    pd.DataFrame({'學校':dept,'流出個數':value}).to_excel('[流出]'+pos+'.xlsx')
    print('[流出]'+pos+'---輸出完成')
    
    ### 列篩選 isin (丟list複合篩選)
    find_In = find_in.iloc[:,0:3].values
    In = [tuple(i) for i in find_In]
    enroll1 = file[file[['排名','考區', '姓名']].apply(tuple, axis = 1).isin(In)]
    
    dept_In = enroll1['申請學校'].value_counts().index
    value_In = enroll1['申請學校'].value_counts().values
    pd.DataFrame({'學校':dept_In,'流入個數':value_In})
    pd.DataFrame({'學校':dept_In,'流入個數':value_In}).to_excel('[流入]'+pos+'.xlsx')
    print('[流入]'+pos+'---輸出完成')

    return real

### step3.資料分析
* 利用step.1讀取之資料跑step.2建立之函式

In [1205]:
for i in range(len(science_name)):
    region_counter(eval('file'+str(i+1)),science_name[i])

----------------------------------------------
【中原大學 企業管理學系服務業管理組】
最後一個錄取: 220
指考人數:0
共有幾人: 56
放棄: 33
入學: 23
[流出]中原大學 企業管理學系服務業管理組---輸出完成
[流入]中原大學 企業管理學系服務業管理組---輸出完成
----------------------------------------------
【中原大學 企業管理學系高科技業管理組】
最後一個錄取: 262
指考仔區域:1
指考人數:1
共有幾人: 69
放棄: 44
入學: 24
[流出]中原大學 企業管理學系高科技業管理組---輸出完成
[流入]中原大學 企業管理學系高科技業管理組---輸出完成
----------------------------------------------
【中原大學 企業管理學系工商管理組】
最後一個錄取: 479
指考仔區域:117
指考人數:1
共有幾人: 121
放棄: 78
入學: 42
[流出]中原大學 企業管理學系工商管理組---輸出完成
[流入]中原大學 企業管理學系工商管理組---輸出完成
----------------------------------------------
【中原大學 國際經營與貿易學系】
最後一個錄取: 653
指考仔區域:15
指考仔區域:16
指考人數:2
共有幾人: 150
放棄: 106
入學: 42
[流出]中原大學 國際經營與貿易學系---輸出完成
[流入]中原大學 國際經營與貿易學系---輸出完成
----------------------------------------------
【中原大學 會計學系】
最後一個錄取: 855
指考仔區域:20
指考仔區域:130
指考仔區域:146
指考仔區域:168
指考仔區域:203
指考仔區域:220
指考人數:6
共有幾人: 224
放棄: 130
入學: 88
[流出]中原大學 會計學系---輸出完成
[流入]中原大學 會計學系---輸出完成
----------------------------------------------
【中原大學 資訊管理學系】
最後一個錄取: 469
指考仔區域:13
指考仔區域:67
指考仔區域:9